# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel, paired_cosine_distances, cosine_similarity
from nltk.corpus import stopwords
import nltk
import streamlit as st
import shap
import streamlit.components.v1 as stc
import pickle
import time
import _pickle as cPickle
# import cPickle as pickle
import gc

In [2]:
merge_dataset = pd.read_csv("/Users/muntaha/Documents/Project4_v2/Game_Recommnedation_system/Data/merged_description.csv", index_col= 0)
game_images = pd.read_csv("/Users/muntaha/Documents/Project4_v2/new_game_data/steam_media_data.csv")


In [3]:
new_data = game_images.rename(index=str, columns={"steam_appid":"appid"})
#merge two datasets
merge_dataset_image = merge_dataset.merge(new_data, on="appid")

## Load TFIDF

In [4]:
tfidf_matrix = pickle.load(open("/Users/muntaha/Documents/Project4_v2/Game_Recommnedation_system/Code/tfidf_matrix.pkl", 'rb'))

# Model

In [54]:
def content_recommender2(name_list,list_game_type ,games, similarities, vote_threshold=1000, rating_threshold=0.7) :
    i = 0
    j = 0
    

        
    recomended_game_name = {}
    game_index = []
    
    top_games = pd.DataFrame()
    
#     find name index that is same game type 
    for name in name_list: #2
        if list(games.iloc[games[games['name']==name_list[i]].index].genres)[0]:
            print(list(games.iloc[games[games['name']==name_list[i]].index].genres)[0])
            game_index.insert(j,games[games['name']==name_list[i]].index)
            i = i+1
            j = j+1
        
        else:
            i = i+1
            continue
    
    for x , y in zip(range(len(name_list)), range(len(game_index))):
        sim_df = pd.DataFrame(
            {'appid': games['appid'],
             'game': games['name'], 
             'game_type': games['genres'], 
             'similarity': np.array(similarities[game_index[y], :].todense()).squeeze(),
             'diversity': 1- np.array(similarities[game_index[y], :].todense()).squeeze(),
             'vote_count': games['total_ratings'],
         'percent_positive_ratings': games['percent_positive_ratings']})
        
        
    
        
        top_games = sim_df[(sim_df['vote_count'] > vote_threshold) & 
                           (sim_df['percent_positive_ratings']>rating_threshold)].sort_values(by='similarity', ascending=False)
        
        
        for t,j in zip(top_games.game, top_games.game_type):
            recomended_game_name[t] = j
        
        
    return recomended_game_name, top_games

In [213]:
def type_based(list_game_type,games,vote_threshold=1000, rating_threshold=0.7):
    index = merge_dataset_image[(merge_dataset_image.genres == list_game_type[0]) & 
                                (merge_dataset_image.total_ratings > vote_threshold) &
                                (merge_dataset_image.percent_positive_ratings > rating_threshold)].sort_values(by='percent_positive_ratings', ascending=False).index
    
    index = index[0:10]
    dicitt = {}
    for x in index:
        dicitt[merge_dataset_image.name.loc[x]] = merge_dataset_image.header_image.loc[x]
    return dicitt

    

### 1) sigmoid_kernel

In [9]:
list_game_name = ['PAYDAY 2', 'Unturned', "Tom Clancy's Rainbow Six® Siege", 'Rust']

top_games = merge_dataset_image[(merge_dataset_image['total_ratings']>1000) & 
                           (merge_dataset_image['percent_positive_ratings']>0.80)].sort_values(by='total_ratings',ascending=False).head(8)

list_of_top_games = list(top_games.name)
listt = []
for x in list_game_name:
    listt.append(top_games[top_games.name == x].index[0])


top_games.drop(listt, axis=0, inplace=True)




2021-11-10 23:46:09.686 INFO    numexpr.utils: NumExpr defaulting to 4 threads.


### 2) cosine_similarity

In [7]:
f = open("/Users/muntaha/Documents/Project4_v2/Game_Recommnedation_system/Code/cosine_similarity_model.pkl", 'rb')
gc.disable()
similarities = cPickle.load(f)
gc.enable()
f.close()

# web streamlite

In [ ]:

def main():
    
    similarities = read_cosine_similarity_model()
    
    page_bg_img = '''
    <style>
    body {
    background-image: url("https://images.alphacoders.com/709/709974.jpg");
    background-size: cover;}
    </style>
    '''
    st.markdown(page_bg_img, unsafe_allow_html=True)
    
    st.title("Game Recommendation App")
    st.markdown("""---""")
    st.write("Choose Games you play")
    
    checkbox1 = st.checkbox("Counter-Strike")
    checkbox2 = st.checkbox("PAYDAY 2")
    time.sleep(2)
    
    #     here all images
    image = merge_dataset_image.head(1).header_image.to_string(index=False).lstrip()
    st.image("https://steamcdn-a.akamaihd.net/steam/apps/10/header.jpg?t=1528733245")
    
    

    
    
    list_game_name,list_game_type = collect_values(checkbox1,checkbox2)
    st.write(list_game_name)
    st.write(list_game_type)
    
    if st.button("Recommend"):
        st.markdown("""---""")
        predect_game(list_game_name,list_game_type,similarities)
        st.write(list_game_name)
        st.write(list_game_type)

                        
        
            
            

def game_type():
    time.sleep(2)
    listt = []
    option = st.selectbox('Choose game type you like:',('Action','Casual','Strategy','Action;RPG'))
    st.write('You selected:', option)
    listt.append(option)
    listt.append(option)

    time.sleep(2)
    return listt

def game_name(g1,g2):
    time.sleep(2)
    listt = []
    if g1:
        listt.append("Counter-Strike")
    if g2:
        listt.append("PAYDAY 2")
    time.sleep(2)
    return listt

    

def predect_game(list_game_name,list_game_type,similarities):
    st.write("All recommended games:")
    st.markdown("""---""")
    similar_games_dict, similar_games_df  = content_recommender2(list_game_name, list_game_type,merge_dataset, similarities, 
                                                                 rating_threshold=0.80)
    time.sleep(1)
    st.write("herre")
    st.write(similar_games_df)
    
    
def collect_values(checkbox1,checkbox2):
    time.sleep(2)
    st.markdown("""---""")
    list_game_name = game_name(checkbox1,checkbox2)
    list_game_type = game_type()  
        
#         if st.button("Recommend"):
#             time.sleep(2)
#             st.markdown("""---""")
#             predect_game(list_game_name,list_game_type)
                 
    return list_game_name,list_game_type

@st.cache(allow_output_mutation = True)
def read_cosine_similarity_model():
    f = open("/Users/muntaha/Documents/Project4_v2/Game_Recommnedation_system/Code/cosine_similarity_model.pkl", 'rb')
    gc.disable()
    similarities = cPickle.load(f)
    gc.enable()
    f.close()
    return similarities



if __name__ == '__main__':
    main()